# Example: General Usage

This example shows the general usage of `PyExperimenter`, from creating an experiment configuration file, over the actual execution of (dummy) experiments, to the extraction of experimental results. 

To execute this notebook you need to install:
```
pip install py_experimenter
pip install scikit-learn
```

## Experiment Configuration File
This notebook shows an example execution of `PyExperimenter` based on an experiment configuration file. Further explanation about the usage of `PyExperimenter` can be found in the [documentation](https://tornede.github.io/py_experimenter/usage.html).

In [1]:
import os

content = """
[PY_EXPERIMENTER]
provider = sqlite 
database = py_experimenter
table = example_general_usage 

keyfields = dataset, cross_validation_splits:int, seed:int, kernel
dataset = iris
cross_validation_splits = 5
seed = 2:6:2 
kernel = linear, poly, rbf, sigmoid

resultfields = pipeline:LONGTEXT, train_f1:DECIMAL, train_accuracy:DECIMAL, test_f1:DECIMAL, test_accuracy:DECIMAL
resultfields.timestamps = false

[CUSTOM] 
path = sample_data
"""
# Create config directory if it does not exist
if not os.path.exists('config'):
    os.mkdir('config')
    
# Create config file
experiment_configuration_file_path = os.path.join('config', 'example_general_usage.cfg')
with open(experiment_configuration_file_path, "w") as f: 
  f.write(content)

## Defining the execution function

Next, the execution of a single experiment has to be defined. Note that this is a dummy example, which contains limited reasonable code. It is meant to show the core functionality of the PyExperimenter. 

The method is called with the parameters, i.e. `keyfields`, of a database entry. The results are meant to be processed to be written into the database, i.e. as `resultfields`. 

In [2]:
import random
import numpy as np

from py_experimenter.result_processor import ResultProcessor

from sklearn.datasets import load_iris
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import cross_validate

def run_ml(parameters: dict, result_processor: ResultProcessor, custom_config: dict):
  seed = parameters['seed']
  random.seed(seed)
  np.random.seed(seed)

  data = load_iris()
  # In case you want to load a file from a path
  # path = os.path.join(custom_config['path'], parameters['dataset'])
  # data = pd.read_csv(path)

  X = data.data
  y = data.target

  model = make_pipeline(StandardScaler(), SVC(kernel=parameters['kernel'], gamma='auto'))  
  result_processor.process_results({
    'pipeline': str(model)
  })

  if parameters['dataset'] != 'iris':
    raise ValueError("Example error")

  scores = cross_validate(model, X, y, 
    cv=parameters['cross_validation_splits'],
    scoring=('accuracy', 'f1_micro'),
    return_train_score=True
  )
  
  result_processor.process_results({
    'train_f1': np.mean(scores['train_f1_micro']),
    'train_accuracy': np.mean(scores['train_accuracy'])
  })

  result_processor.process_results({
    'test_f1': np.mean(scores['test_f1_micro']),
    'test_accuracy': np.mean(scores['test_accuracy'])
  })

## Executing PyExperimenter

The actual execution of the PyExperimenter is done in multiple steps. 

### Initialize PyExperimenter
The PyExperimenter is initialized with the previously created configuration file. Additionally, `PyExperimenter` is given a `name`, i.e. job id, which is especially useful for parallel executions of multiple experiments on HPC. 

In [3]:
from py_experimenter.experimenter import PyExperimenter

experimenter = PyExperimenter(experiment_configuration_file_path=experiment_configuration_file_path, name='example_notebook')

### Fill Table

The table is filled based on the above created configuration file with `fill_table_from_config()`. Therefore, the cartesian product of all keyfields makes up the content of the table. Additionally, a custom defined row, i.e. a custom defined keyfield tuple, is added with `fill_table_with_rows()`. 

Note that the table can easily be obtained as `pandas.Dataframe` via `experimenter.get_table()`.

In [4]:
experimenter.fill_table_from_config()

experimenter.fill_table_with_rows(rows=[
      {'dataset': 'error_dataset', 'cross_validation_splits': 3, 'seed': 42, 'kernel':'linear'}])

# showing database table
experimenter.get_table()

,ID,dataset,cross_validation_splits,seed,kernel,creation_date,status,start_date,name,machine,pipeline,train_f1,train_accuracy,test_f1,test_accuracy,end_date,error
0,1,iris,5,2,linear,2023-03-31 20:47:09,created,None,None,None,None,None,None,None,None,None,None
1,2,iris,5,4,linear,2023-03-31 20:47:09,created,None,None,None,None,None,None,None,None,None,None
2,3,iris,5,6,linear,2023-03-31 20:47:09,created,None,None,None,None,None,None,None,None,None,None
3,4,iris,5,2,poly,2023-03-31 20:47:09,created,None,None,None,None,None,None,None,None,None,None
4,5,iris,5,4,poly,2023-03-31 20:47:09,created,None,None,None,None,None,None,None,None,None,None
5,6,iris,5,6,poly,2023-03-31 20:47:09,created,None,None,None,None,None,None,None,None,None,None
6,7,iris,5,2,rbf,2023-03-31 20:47:09,created,None,None,None,None,None,None,None,None,None,None
7,8,iris,5,4,rbf,2023-03-31 20:47:09,created,None,None,None,None,None,None,None,None,None,None
8,9,iris,5,6,rbf,2023-03-31 20:47:09,created,None,None,None,None,None,None,None,None,None,None
9,10,iris,5,2,sigmoid,2023-03-31 20:47:09,created,None,None,None,None,None,None,None,None,None,None


### Execute PyExperimenter
All experiments are executed one after the other by the same `PyExperimenter` due to `max_experiments=-1`. If just a single one or a predifined number of experiments should be executed, the `-1` has to be replaced by the according amount

The first parameter, i.e. `run_ml`, relates to the actual method that should be executed with the given keyfields of the table. 

In [5]:
experimenter.execute(run_ml, max_experiments=-1)

# showing database table
experimenter.get_table() 

ERROR:root:Traceback (most recent call last):
  File "/Users/tanja/Development/py_experimenter/py_experimenter/experimenter.py", line 370, in _execution_wrapper
    experiment_function(keyfield_values, result_processor, custom_fields)
  File "/var/folders/8l/vv83bgdn0zg97jp8pbsfxq040000gn/T/ipykernel_13290/1244630566.py", line 31, in run_ml
    raise ValueError("Example error")
ValueError: Example error



,ID,dataset,cross_validation_splits,seed,kernel,creation_date,status,start_date,name,machine,pipeline,train_f1,train_accuracy,test_f1,test_accuracy,end_date,error
0,1,iris,5,2,linear,2023-03-31 20:47:09,done,2023-03-31 20:47:09,example_notebook,MacBook-Pro-von-Tanja.local,"Pipeline(steps=[('standardscaler', StandardSca...",0.971667,0.971667,0.966667,0.966667,2023-03-31 20:47:09,None
1,2,iris,5,4,linear,2023-03-31 20:47:09,done,2023-03-31 20:47:09,example_notebook,MacBook-Pro-von-Tanja.local,"Pipeline(steps=[('standardscaler', StandardSca...",0.971667,0.971667,0.966667,0.966667,2023-03-31 20:47:09,None
2,3,iris,5,6,linear,2023-03-31 20:47:09,done,2023-03-31 20:47:09,example_notebook,MacBook-Pro-von-Tanja.local,"Pipeline(steps=[('standardscaler', StandardSca...",0.971667,0.971667,0.966667,0.966667,2023-03-31 20:47:09,None
3,4,iris,5,2,poly,2023-03-31 20:47:09,done,2023-03-31 20:47:09,example_notebook,MacBook-Pro-von-Tanja.local,"Pipeline(steps=[('standardscaler', StandardSca...",0.936667,0.936667,0.933333,0.933333,2023-03-31 20:47:09,None
4,5,iris,5,4,poly,2023-03-31 20:47:09,done,2023-03-31 20:47:09,example_notebook,MacBook-Pro-von-Tanja.local,"Pipeline(steps=[('standardscaler', StandardSca...",0.936667,0.936667,0.933333,0.933333,2023-03-31 20:47:09,None
5,6,iris,5,6,poly,2023-03-31 20:47:09,done,2023-03-31 20:47:09,example_notebook,MacBook-Pro-von-Tanja.local,"Pipeline(steps=[('standardscaler', StandardSca...",0.936667,0.936667,0.933333,0.933333,2023-03-31 20:47:09,None
6,7,iris,5,2,rbf,2023-03-31 20:47:09,done,2023-03-31 20:47:09,example_notebook,MacBook-Pro-von-Tanja.local,"Pipeline(steps=[('standardscaler', StandardSca...",0.975000,0.975000,0.966667,0.966667,2023-03-31 20:47:09,None
7,8,iris,5,4,rbf,2023-03-31 20:47:09,done,2023-03-31 20:47:09,example_notebook,MacBook-Pro-von-Tanja.local,"Pipeline(steps=[('standardscaler', StandardSca...",0.975000,0.975000,0.966667,0.966667,2023-03-31 20:47:09,None
8,9,iris,5,6,rbf,2023-03-31 20:47:09,done,2023-03-31 20:47:09,example_notebook,MacBook-Pro-von-Tanja.local,"Pipeline(steps=[('standardscaler', StandardSca...",0.975000,0.975000,0.966667,0.966667,2023-03-31 20:47:09,None
9,10,iris,5,2,sigmoid,2023-03-31 20:47:09,done,2023-03-31 20:47:09,example_notebook,MacBook-Pro-von-Tanja.local,"Pipeline(steps=[('standardscaler', StandardSca...",0.896667,0.896667,0.893333,0.893333,2023-03-31 20:47:09,None


### Restart Failed Experiments

As experiments fail at some time, those experiments were reset for another try with `reset_experiments()`. The `status` describes which table rows should be replace. In this example all failed experiments, i.e. having `status==error`, are reset.

In [6]:
experimenter.reset_experiments('error')

# experiments can also be reset based on multiple status by simply passing a list of status
# experimenter.reset_experiments('error', 'done')

# showing database table
experimenter.get_table() 

,ID,dataset,cross_validation_splits,seed,kernel,creation_date,status,start_date,name,machine,pipeline,train_f1,train_accuracy,test_f1,test_accuracy,end_date,error
0,1,iris,5,2,linear,2023-03-31 20:47:09,done,2023-03-31 20:47:09,example_notebook,MacBook-Pro-von-Tanja.local,"Pipeline(steps=[('standardscaler', StandardSca...",0.971667,0.971667,0.966667,0.966667,2023-03-31 20:47:09,None
1,2,iris,5,4,linear,2023-03-31 20:47:09,done,2023-03-31 20:47:09,example_notebook,MacBook-Pro-von-Tanja.local,"Pipeline(steps=[('standardscaler', StandardSca...",0.971667,0.971667,0.966667,0.966667,2023-03-31 20:47:09,None
2,3,iris,5,6,linear,2023-03-31 20:47:09,done,2023-03-31 20:47:09,example_notebook,MacBook-Pro-von-Tanja.local,"Pipeline(steps=[('standardscaler', StandardSca...",0.971667,0.971667,0.966667,0.966667,2023-03-31 20:47:09,None
3,4,iris,5,2,poly,2023-03-31 20:47:09,done,2023-03-31 20:47:09,example_notebook,MacBook-Pro-von-Tanja.local,"Pipeline(steps=[('standardscaler', StandardSca...",0.936667,0.936667,0.933333,0.933333,2023-03-31 20:47:09,None
4,5,iris,5,4,poly,2023-03-31 20:47:09,done,2023-03-31 20:47:09,example_notebook,MacBook-Pro-von-Tanja.local,"Pipeline(steps=[('standardscaler', StandardSca...",0.936667,0.936667,0.933333,0.933333,2023-03-31 20:47:09,None
5,6,iris,5,6,poly,2023-03-31 20:47:09,done,2023-03-31 20:47:09,example_notebook,MacBook-Pro-von-Tanja.local,"Pipeline(steps=[('standardscaler', StandardSca...",0.936667,0.936667,0.933333,0.933333,2023-03-31 20:47:09,None
6,7,iris,5,2,rbf,2023-03-31 20:47:09,done,2023-03-31 20:47:09,example_notebook,MacBook-Pro-von-Tanja.local,"Pipeline(steps=[('standardscaler', StandardSca...",0.975000,0.975000,0.966667,0.966667,2023-03-31 20:47:09,None
7,8,iris,5,4,rbf,2023-03-31 20:47:09,done,2023-03-31 20:47:09,example_notebook,MacBook-Pro-von-Tanja.local,"Pipeline(steps=[('standardscaler', StandardSca...",0.975000,0.975000,0.966667,0.966667,2023-03-31 20:47:09,None
8,9,iris,5,6,rbf,2023-03-31 20:47:09,done,2023-03-31 20:47:09,example_notebook,MacBook-Pro-von-Tanja.local,"Pipeline(steps=[('standardscaler', StandardSca...",0.975000,0.975000,0.966667,0.966667,2023-03-31 20:47:09,None
9,10,iris,5,2,sigmoid,2023-03-31 20:47:09,done,2023-03-31 20:47:09,example_notebook,MacBook-Pro-von-Tanja.local,"Pipeline(steps=[('standardscaler', StandardSca...",0.896667,0.896667,0.893333,0.893333,2023-03-31 20:47:09,None


After the reset of failed experiments, they can be executed again as described above. 

In [7]:
experimenter.execute(run_ml, max_experiments=-1)

# showing database table
experimenter.get_table() 

ERROR:root:Traceback (most recent call last):
  File "/Users/tanja/Development/py_experimenter/py_experimenter/experimenter.py", line 370, in _execution_wrapper
    experiment_function(keyfield_values, result_processor, custom_fields)
  File "/var/folders/8l/vv83bgdn0zg97jp8pbsfxq040000gn/T/ipykernel_13290/1244630566.py", line 31, in run_ml
    raise ValueError("Example error")
ValueError: Example error



,ID,dataset,cross_validation_splits,seed,kernel,creation_date,status,start_date,name,machine,pipeline,train_f1,train_accuracy,test_f1,test_accuracy,end_date,error
0,1,iris,5,2,linear,2023-03-31 20:47:09,done,2023-03-31 20:47:09,example_notebook,MacBook-Pro-von-Tanja.local,"Pipeline(steps=[('standardscaler', StandardSca...",0.971667,0.971667,0.966667,0.966667,2023-03-31 20:47:09,None
1,2,iris,5,4,linear,2023-03-31 20:47:09,done,2023-03-31 20:47:09,example_notebook,MacBook-Pro-von-Tanja.local,"Pipeline(steps=[('standardscaler', StandardSca...",0.971667,0.971667,0.966667,0.966667,2023-03-31 20:47:09,None
2,3,iris,5,6,linear,2023-03-31 20:47:09,done,2023-03-31 20:47:09,example_notebook,MacBook-Pro-von-Tanja.local,"Pipeline(steps=[('standardscaler', StandardSca...",0.971667,0.971667,0.966667,0.966667,2023-03-31 20:47:09,None
3,4,iris,5,2,poly,2023-03-31 20:47:09,done,2023-03-31 20:47:09,example_notebook,MacBook-Pro-von-Tanja.local,"Pipeline(steps=[('standardscaler', StandardSca...",0.936667,0.936667,0.933333,0.933333,2023-03-31 20:47:09,None
4,5,iris,5,4,poly,2023-03-31 20:47:09,done,2023-03-31 20:47:09,example_notebook,MacBook-Pro-von-Tanja.local,"Pipeline(steps=[('standardscaler', StandardSca...",0.936667,0.936667,0.933333,0.933333,2023-03-31 20:47:09,None
5,6,iris,5,6,poly,2023-03-31 20:47:09,done,2023-03-31 20:47:09,example_notebook,MacBook-Pro-von-Tanja.local,"Pipeline(steps=[('standardscaler', StandardSca...",0.936667,0.936667,0.933333,0.933333,2023-03-31 20:47:09,None
6,7,iris,5,2,rbf,2023-03-31 20:47:09,done,2023-03-31 20:47:09,example_notebook,MacBook-Pro-von-Tanja.local,"Pipeline(steps=[('standardscaler', StandardSca...",0.975000,0.975000,0.966667,0.966667,2023-03-31 20:47:09,None
7,8,iris,5,4,rbf,2023-03-31 20:47:09,done,2023-03-31 20:47:09,example_notebook,MacBook-Pro-von-Tanja.local,"Pipeline(steps=[('standardscaler', StandardSca...",0.975000,0.975000,0.966667,0.966667,2023-03-31 20:47:09,None
8,9,iris,5,6,rbf,2023-03-31 20:47:09,done,2023-03-31 20:47:09,example_notebook,MacBook-Pro-von-Tanja.local,"Pipeline(steps=[('standardscaler', StandardSca...",0.975000,0.975000,0.966667,0.966667,2023-03-31 20:47:09,None
9,10,iris,5,2,sigmoid,2023-03-31 20:47:09,done,2023-03-31 20:47:09,example_notebook,MacBook-Pro-von-Tanja.local,"Pipeline(steps=[('standardscaler', StandardSca...",0.896667,0.896667,0.893333,0.893333,2023-03-31 20:47:09,None


### Generating Result Table


The table containes single experiment results. Those can be aggregated, e.g. to generate the mean over all seeds. 

In [8]:
result_table_agg = experimenter.get_table().groupby(['dataset']).mean(numeric_only = True)
result_table_agg

,ID,cross_validation_splits,seed,train_f1,train_accuracy,test_f1,test_accuracy
dataset,,,,,,,
error_dataset,14.0,3.0,42.0,NaN,NaN,NaN,NaN
iris,6.5,5.0,4.0,0.945,0.945,0.94,0.94


### Printing LaTex Table

As `pandas.Dataframe`s can easily be printed as LaTex table, here is an example code for one of the above result columns. 

In [9]:
print(result_table_agg[['test_f1']].style.to_latex())

\begin{tabular}{lr}
 & test_f1 \\
dataset &  \\
error_dataset & nan \\
iris & 0.940000 \\
\end{tabular}

